В этом ноубуке используются обeченные тестовые модели, с их помощью происходит сэмплирование и далее закодированные категориальные переменные переводятся в изначальные. После чего замеряется качество.

Способ нормализации: quantile transformation для числовых признаков и standart scale для категориальных OHE

Добавление шума: ко всем признакам

In [1]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ON_QnSC.tabddpm_ON_QnSC import *

from tqdm.notebook import tqdm

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

### Подготовка

In [2]:
dataname = 'adult_ON_QnSC'
model_short = 'tabddpm_ON_QnSC'

In [3]:
if torch.cuda.is_available():
    device = f'cuda'
else:
    device = 'cpu'
CONFIG.add_arg('device', device)
print(f"Device: {device}")

CONFIG.add_arg('dataname', dataname)
CONFIG.add_arg('real_path',
                   f'./synthetic/{dataname}/initial_real.csv')
CONFIG.add_arg('test_path',
                   f'./synthetic/{dataname}/initial_test.csv')

CONFIG.add_arg('info_path',
                   f'./data/{dataname}/initial_info.json')

Device: cuda


In [4]:
model_save_path_hist = []
for sigma in [0, 0.001, 0.01, 0.1, 0.25, 0.5]:
    model_save_path=f"./models/{model_short}/ckpt/{model_short}_const_{str(sigma).replace('.', '_')}"
    model_save_path_hist.append(model_save_path)

for i, sigmas in enumerate(range(10)):
    model_save_path=f"./models/{model_short}/ckpt/{model_short}_mult_{i}"
    model_save_path_hist.append(model_save_path)

In [5]:
from pickle import dump, load
with open(f'data/{dataname}/normalizers.json', 'r') as f:
        normalizers = json.load(f)
normalizers['num_normalizer'] = load(open(f'./data/{dataname}/num_normalizer_{dataname}.pkl', 'rb'))
normalizers['cat_normalizer'] = load(open(f'./data/{dataname}/cat_normalizer_{dataname}.pkl', 'rb'))
print(f"normalizers received from `data/{dataname}` folder")

normalizers received from `data/adult_ON_QnSC` folder


In [6]:
normalizers

{'len_num_prev': 6,
 'len_cat_prev': 102,
 'len_target_prev': 1,
 'num_normalizer': QuantileTransformer(output_distribution='normal', random_state=0,
                     subsample=1000000000),
 'cat_normalizer': StandardScaler()}

### Sample + Eval

In [7]:
CONFIG.get_all_args()

{'dataname': 'adult_ON_QnSC',
 'method': 'tabddpm_ON_QnSC_const_0',
 'device': 'cuda',
 'mode': 'train',
 'train': 1,
 'sample_save_path': 'synthetic/adult_ON_QnSC/tabddpm_ON_QnSC_const_0.csv',
 'sigma_scheduller_name': 'constant',
 'sigma_value': 0.001,
 'num_noise': 103,
 'real_path': './synthetic/adult_ON_QnSC/initial_real.csv',
 'test_path': './synthetic/adult_ON_QnSC/initial_test.csv',
 'info_path': './data/adult_ON_QnSC/initial_info.json',
 'save_path': './synthetic/adult_ON_QnSC/initial_tabddpm_ON_QnSC_const_0.csv'}

In [ ]:
overall_metrics = {}

for model_save_path in model_save_path_hist:
    model_name = model_save_path.split('/')[-1]
    sample_save_path = f'./synthetic/{dataname}/{model_name}.csv'

    CONFIG.add_arg('method', model_name)
    CONFIG.add_arg('sample_save_path',
                       f"synthetic/{CONFIG.get_arg('dataname')}/{CONFIG.get_arg('method')}.csv")

    overall_metrics[model_name] = {}

    print(model_save_path, model_name, sample_save_path)

    tabddpm_noise_ohe = TabDDPM_OHE_Noise_QnSC(CONFIG, model_save_path=model_save_path, sigmas=None)
    tabddpm_noise_ohe.sample(sample_save_path=sample_save_path)
    postsample_OHE(dataname, f'./synthetic/{dataname}/initial_{model_name}.csv', normalizers=normalizers)

    # подсчет метрик
    try:
        overall_metrics[model_name]['base_metrics'] = calculate_base_metrics(make_binary=True, value=' >50K') # ошибка
    except Exception as e:
        overall_metrics[model_name]['base_metrics'] = np.nan
    overall_metrics[model_name]['similarity'] = calculate_similarity()
    overall_metrics[model_name]['mle'] = calculate_mle()
    overall_metrics[model_name]['detection'] = calculate_detection()
    overall_metrics[model_name]['DCR'] = calculate_DCR()
    overall_metrics[model_name]['quality'] = calculate_alpha_beta()

In [11]:
final_metrics_table = []
for m in overall_metrics.keys():
    # Сбор таблички результатов
    tmp = pd.DataFrame([{'Model':model_short, 'Type':m, 'Data':dataname}])
    tmp.columns = pd.MultiIndex.from_tuples([('', i) for i in tmp.columns])
    result = [tmp]
    
    for metric_group in overall_metrics[m].keys():
        tmp = pd.DataFrame([overall_metrics[m][metric_group]])
        tmp.columns = pd.MultiIndex.from_tuples([(metric_group, i) for i in tmp.columns])
        result.append(tmp)
    result = pd.concat(result, axis = 1)
    final_metrics_table.append(result)
    
final_metrics_table = pd.concat(final_metrics_table)

In [12]:
final_metrics_table.drop(columns=('base_metrics', 0))

\
             Model                         Type           Data   
0  tabddpm_ON_QnSC      tabddpm_ON_QnSC_const_0  adult_ON_QnSC   
0  tabddpm_ON_QnSC  tabddpm_ON_QnSC_const_0_001  adult_ON_QnSC   
0  tabddpm_ON_QnSC   tabddpm_ON_QnSC_const_0_01  adult_ON_QnSC   
0  tabddpm_ON_QnSC    tabddpm_ON_QnSC_const_0_1  adult_ON_QnSC   
0  tabddpm_ON_QnSC   tabddpm_ON_QnSC_const_0_25  adult_ON_QnSC   
0  tabddpm_ON_QnSC    tabddpm_ON_QnSC_const_0_5  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_0  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_1  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_2  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_3  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_4  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_5  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_6  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_7  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_8  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_9  adult_ON_QnSC   

              similarity                              \
  Column Shapes Score, % Column Pair Trends Score, %   
0              97.761944                   95.860595   
0              97.912022                   95.827279   
0              98.227327                   96.069651   
0              97.577879                   95.433992   
0              98.005794                   96.362806   
0              97.806988                   95.469228   
0              94.681367                   90.343251   
0              96.826469                   93.142738   
0              97.720381                   93.949490   
0              97.769110                   94.897559   
0              37.647902                   27.919564   
0              98.133759                   96.582250   
0              98.028726                   96.325048   
0              97.919802                   96.069353   
0              97.980406                   96.014838   
0              97.863497                   95.953344   

                              \
  Overall Score (Average), %   
0                  96.811270   
0                  96.869650   
0                  97.148489   
0                  96.505936   
0                  97.184300   
0                  96.638108   
0                  92.512309   
0                  94.984604   
0                  95.834936   
0                  96.333335   
0                  32.783733   
0                  97.358005   
0                  97.176887   
0                  96.994578   
0                  96.997622   
0                  96.908420   

                                                       \
  Error rate (%) of column-wise density estimation, %   
0                                            2.238056   
0                                            2.087978   
0                                            1.772673   
0                                            2.422121   
0                                            1.994206   
0                                            2.193012   
0                                            5.318633   
0                                            3.173531   
0                                            2.279619   
0                                            2.230890   
0                                           62.352098   
0                                            1.866241   
0                                            1.971274   
0                                            2.080198   
0                                            2.019594   
0                                            2.136503   

                                                           \
  Error rate (%) of column-wise density estimation std, %   
0                                                1.406972   
0                                                1.216163   
0                 

In [17]:
pd.set_option('display.max_columns', None)
final_metrics_table.round(3).drop(columns=('base_metrics', 0)).sort_values([('similarity', 'Overall Score (Average), %')],
                                                                          ascending=False)

\
             Model                         Type           Data   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_5  adult_ON_QnSC   
0  tabddpm_ON_QnSC   tabddpm_ON_QnSC_const_0_25  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_6  adult_ON_QnSC   
0  tabddpm_ON_QnSC   tabddpm_ON_QnSC_const_0_01  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_8  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_7  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_9  adult_ON_QnSC   
0  tabddpm_ON_QnSC  tabddpm_ON_QnSC_const_0_001  adult_ON_QnSC   
0  tabddpm_ON_QnSC      tabddpm_ON_QnSC_const_0  adult_ON_QnSC   
0  tabddpm_ON_QnSC    tabddpm_ON_QnSC_const_0_5  adult_ON_QnSC   
0  tabddpm_ON_QnSC    tabddpm_ON_QnSC_const_0_1  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_3  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_2  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_1  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_0  adult_ON_QnSC   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_mult_4  adult_ON_QnSC   

              similarity                              \
  Column Shapes Score, % Column Pair Trends Score, %   
0                 98.134                      96.582   
0                 98.006                      96.363   
0                 98.029                      96.325   
0                 98.227                      96.070   
0                 97.980                      96.015   
0                 97.920                      96.069   
0                 97.863                      95.953   
0                 97.912                      95.827   
0                 97.762                      95.861   
0                 97.807                      95.469   
0                 97.578                      95.434   
0                 97.769                      94.898   
0                 97.720                      93.949   
0                 96.826                      93.143   
0                 94.681                      90.343   
0                 37.648                      27.920   

                              \
  Overall Score (Average), %   
0                     97.358   
0                     97.184   
0                     97.177   
0                     97.148   
0                     96.998   
0                     96.995   
0                     96.908   
0                     96.870   
0                     96.811   
0                     96.638   
0                     96.506   
0                     96.333   
0                     95.835   
0                     94.985   
0                     92.512   
0                     32.784   

                                                       \
  Error rate (%) of column-wise density estimation, %   
0                                               1.866   
0                                               1.994   
0                                               1.971   
0                                               1.773   
0                                               2.020   
0                                               2.080   
0                                               2.137   
0                                               2.088   
0                                               2.238   
0                                               2.193   
0                                               2.422   
0                                               2.231   
0                                               2.280   
0                                               3.174   
0                                               5.319   
0                                              62.352   

                                                           \
  Error rate (%) of column-wise density estimation std, %   
0                                                   1.056   
0                                                   1.287   
0                 

In [15]:
import os
if not os.path.exists(f'./eval/total/{dataname}'):
    os.makedirs(f'./eval/total/{dataname}')
final_metrics_table.to_csv(f'./eval/total/{dataname}/{model_short}_final_metrcs_table.csv', index=False)